# Evaluation of the anomaly detection

This notebook evaluates trained autoencoders

---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from data_loading import TimeSeriesWithAnoms
from params import TcnAeParams, tcn_ae_params
from evaluation import EvaluationMetric, PrecisionAtK, F1Score, evaluate_model
from train_models import prepare_data
from detectors import TcnAeDetector

### Load and prepare data

We will load all time series and normalize them (we use the same function as in the training script)

In [2]:
all_series = prepare_data()
all_series[0]

TimeSeriesWithAnoms(name='SinusRW_Length_112000_AnomalyL_200_AnomalyN_60_NoisePerc_0', values=array([0.78267852, 0.77670079, 0.77072901, ..., 0.54603821, 0.54006643,
       0.5340887 ]), annotations=[(1070, 1270), (3245, 3445), (5200, 5400), (6245, 6445), (7468, 7668), (10367, 10567), (11337, 11537), (13161, 13361), (13686, 13886), (14208, 14408), (16211, 16411), (19490, 19690), (23336, 23536), (24398, 24598), (28747, 28947), (30869, 31069), (31927, 32127), (33280, 33480), (35285, 35485), (38473, 38673), (43829, 44029), (45524, 45724), (48857, 49057), (49095, 49295), (50200, 50400), (51410, 51610), (52926, 53126), (56144, 56344), (60167, 60367), (62178, 62378), (63715, 63915), (63976, 64176), (66221, 66421), (69191, 69391), (69961, 70161), (72081, 72281), (72694, 72894), (73269, 73469), (75091, 75291), (76875, 77075), (77531, 77731), (84824, 85024), (85373, 85573), (87237, 87437), (89419, 89619), (90802, 91002), (94382, 94582), (95438, 95638), (95713, 95913), (96599, 96799), (97139, 97

In [3]:
# number of time series
len(all_series)

14

### Run anomaly detection and get metrics

We will perform the following operations, for each time series:

- load trained model on the series
- get pointwise reconstruction errors on the series
- extract sliding windows from the errors and detect anomalous windows
- calculate metrics

In [4]:
def evaluate_tcnae(
    all_series: list[TimeSeriesWithAnoms],
    metrics: list[EvaluationMetric],
    params: TcnAeParams,
) -> dict[str, dict[str, float]]:
    """Loads the model and evaluates detection for each series

    Returns dict:
    {series_name -> {metric_name -> metric_value}}
    """
    series_metrics_dict: dict[str, dict[str, float]] = {}
    for series in tqdm(all_series):
        detector = TcnAeDetector.load(series, params)
        series_metrics_dict[series.name] = evaluate_model(
            series, detector, metrics
        )
    return series_metrics_dict

In [5]:
metrics = [PrecisionAtK(), F1Score()]

tcnae_metrics = evaluate_tcnae(
    all_series=all_series,
    metrics=metrics,
    params=tcn_ae_params,
)

  0%|          | 0/14 [00:00<?, ?it/s]